# Modified 3D PMRNN

## Module Imports

In [5]:
from Reconstruction import Reconstruction
from CONFIGURATION import CONFIGURATION
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
tf.compat.v1.disable_eager_execution()
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

In [6]:
# ignorable
print("TensorFlow version:", tf.__version__)
print("cv2 version:", cv2.__version__)
print("np version:", np.__version__)

TensorFlow version: 2.7.0
cv2 version: 4.5.5
np version: 1.19.5


In [7]:
# ignorable
import sklearn
import matplotlib
print("matplotlib version:", matplotlib.__version__)
print("sklearn version:", sklearn.__version__)

matplotlib version: 3.6.3
sklearn version: 1.2.2


In [4]:
from tensorflow.keras import layers, losses  # ,Sequential,metrics
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import *
# from tensorflow.keras.layers import Layer
# from tensorflow.keras import optimizers
# from tensorflow.keras.optimizers.experimental import SGD
# from tensorflow.image import ssim

## Load new Data

### reading data

In [ ]:
import Utils
image_data = Utils.read_data("Berea_2d25um_binary.raw")
image_data.shape

### Voxel extraction 

In [ ]:
voxels = Utils.extract_subvolumes(image_data)
voxels.shape

In [ ]:
del image_data

### Train and test data extraction

In [ ]:
X_train = voxels[:-6]
X_test = voxels[-6:]

In [ ]:
del voxels

In [ ]:
len(X_train)

### Save Traited data for uses

In [ ]:
with open('X_train.npy', 'wb') as f:
    np.save(f, X_train)

In [ ]:
with open('X_test.npy', 'wb') as f:
    np.save(f, X_test)

## TensorFlow GPU setup

In [5]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
device = tf.config.list_physical_devices("GPU")
print(device)  # Check if GPU devices are visible
# Check if TensorFlow is built with CUDA support
print(tf.test.is_built_with_cuda())
print(trt.trt_utils._pywrap_py_utils.get_linked_tensorrt_version())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
True
(7, 2, 2)


In [6]:
# only on weak GPU
tf.config.experimental.set_memory_growth(device[0], True)
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [7]:
tf.keras.backend.clear_session()

## Load Traited data for training (voxels)

In [8]:
X_train = np.load('X_train.npy')
X_train.shape

(58, 250, 256, 256, 1)

In [9]:
# for test only
X_train = X_train[:4]
with open('X_train_test.npy', 'wb') as f:
    np.save(f, X_train)
X_train.shape

(4, 250, 256, 256, 1)

In [5]:
X_train = np.load('X_train_test.npy')
X_train.shape

(1, 250, 256, 256, 1)

### idk

In [9]:
learnedVoxels = []
inferenceVoxels = []
for voxel in X_train:
    learnedVoxels.append(voxel[:-1])
    inferenceVoxels.append(voxel[1:])

# learnedVoxels = np.array(learnedVoxels)
# inferenceVoxels = np.array(inferenceVoxels)

## Training 

### Imports

In [10]:
from mealpy.swarm_based import GWO
from sklearn.preprocessing import LabelEncoder
import time
import pickle
from datetime import datetime

In [11]:
import mealpy
mealpy.__version__

'2.4.2'

### Lables

In [12]:
class Timer():
    def __init__(self, title=" ", p={}) -> None:
        self.title = title
        self.started = time.time()
        self.params = p
        print(f"{self.title} Started at : {datetime.fromtimestamp(self.started)}")
        self.ended = None

    def End(self):
        self.ended = time.time()
        print(f"{self.title} Ended at : {datetime.fromtimestamp(self.ended)} and it took {self.ended-self.started}s")

    def setParams(self, name, data):
        self.params[name] = data

    def toDuration(self):
        if self.ended == None:
            return -1
        return self.ended-self.started

    def __str__(self) -> str:
        return f"{self.title} Started at : {datetime.fromtimestamp(self.started)} | Ended at : {datetime.fromtimestamp(self.ended)} and it took {self.ended-self.started}s"


class TrainingData(Timer):
    def __init__(self, title=" ", p={}) -> None:
        super().__init__(title=title, p=p)
        self.samples = []

    def SampleStarted(self, title=" ") -> Timer:
        timer = Timer(title)
        self.samples.append(timer)
        return timer

    def __str__(self) -> str:
        return super().__str__()+" "+str(self.params)

In [13]:
OPT_ENCODER = LabelEncoder()
# OPT_ENCODER.fit(['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam'])
OPT_ENCODER.fit(['SGD', 'RMSprop', 'Adagrad', 'Adam'])
trainings_duration = []
trainings_epochs = []
current_iteration = 0

### Solution Decodation function

In [14]:
def decode_solution(solution):
    batch_size = 2**int(solution[0])

    learning_rate = solution[1]

    opt_int = int(solution[2])
    opt = OPT_ENCODER.inverse_transform([opt_int])[0]

    epoch = 10 * int(solution[3])

    num_filters = int(solution[4])

    latent_space_dim = 2**int(solution[5])

    reduced_dim = 2**int(solution[6])

    return [batch_size, learning_rate, opt, epoch, num_filters, latent_space_dim, reduced_dim]

### Objective Function

In [15]:
""" import random
# the range of float values
start = 0.001
end = 0.8 """
current_training=0

def objective_function(solution):
    tf.keras.backend.clear_session()
    global current_training
    print("\n###########################################################\n")
    print(f"\n--------------------------{current_iteration}/{current_training}--------------------------------\n")
    print("\n###########################################################\n")
    batch_size, learning_rate, opt, epoch, num_filters, latent_space_dim, reduced_dim = decode_solution(solution)
    reconstruction = Reconstruction(inputShape=CONFIGURATION["INPUT_SHAPE"],
                                    latent_space_dim=latent_space_dim,
                                    reducedDimension=reduced_dim,
                                    num_conv_layers=num_filters,
                                    learning_rate=learning_rate,
                                    batch_size=batch_size,
                                    epochs=epoch,
                                    opt=opt,
                                    )
    # reconstruction.summary()
    reconstruction.compile()
    
    params = {
        "inputShape": CONFIGURATION["INPUT_SHAPE"],
        "latent_space_dim": latent_space_dim,
        "reducedDimension": reduced_dim,
        "num_conv_layers": num_filters,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "epochs": epoch,
        "opt": opt,
        "finished":False
    }
    trainingTime = TrainingData("Epoch Training", params)
    trainings_duration.append(trainingTime)
    training_loss=1
    for i in range(len(learnedVoxels)):
        print(f"---Currently working one voxel : {i +1}, voxels left : {len(learnedVoxels)-(i+1)} ")
        sampleTime = trainingTime.SampleStarted(f"Voxel Training {i+1}")
        try:
            # Code that may raise an OOM error
            training_result=reconstruction.train(learnedVoxels[i], inferenceVoxels[i])
        except MemoryError as e:
            # Code to handle the OOM error
            trainingTime.setParams("OMM_Errored",str(e))
        except Exception as e:
            trainingTime.setParams("Error",str(e))

        sampleTime.End()
        training_loss=training_result.history['loss'][-1]
        if training_loss >0.1:
            trainingTime.setParams("Bad_params",str(e))
            break

    trainingTime.setParams("loss",training_loss)
    trainingTime.setParams("finished",True)
    trainingTime.End()

    current_training+=1
    
    """ reconstruction.save( save_folder=f"results/model_{batch_size}_{learning_rate}_{opt}_{epoch}_{num_filters}_{latent_space_dim}_{reduced_dim}") """
    #return min(histories, key=lambda x: float('inf') if x is None else x)
    return training_loss

### Configurations

In [16]:
LB = [3, 0.001, 0, 2,  4,  6, 7]
UB = [5.99, 0.01, 3.99, 4.99,  8.99,  8.99, 9.99]

problem_dict = {
    "fit_func": objective_function,
    "lb": LB,
    "ub": UB,
    "minmax": "min",
    "verbose": True,
}
metaheuristic_configuration = {
    "epoch": 5,
    "pop_size": 10
}
# save session for x epoch of the training
save_interval = 1
session_file = 'gwo_session.pkl'

### Lunching the metaheuristic

In [17]:
# Check if a session file exists
if os.path.exists(session_file):
    # Load the session and resume from the saved state
    with open(session_file, 'rb') as f:
        session_data = pickle.load(f)
    model = session_data['model']
    best_solution = session_data['best_solution']
    current_iteration = session_data['current_iteration']
    print(best_solution)
    print("Resuming from iteration", current_iteration)
    # raise ValueError("i just wanted to stop")
else:
    # Create a new GWO optimization session
    model = GWO.BaseGWO(
        problem_dict, epoch=metaheuristic_configuration["epoch"], pop_size=metaheuristic_configuration["pop_size"])
    best_solution = None
    current_iteration = 0
    trainings_epochs.append(
        {"epoch": current_iteration, "trainings": trainings_duration, "best_solution": best_solution})
    trainings_duration = []

{'fit': 0.01349127304975886, 'position': array([3.92447043e+00, 4.53517989e-03, 2.24030586e+00, 4.35225047e+00,
       8.53341056e+00, 6.79257622e+00, 7.66029412e+00])}
Resuming from iteration 2


In [18]:
# Run the optimization loop
# while current_iteration < metaheuristic_configuration["epoch"]:
# Perform GWO iteration
best_position, best_fitness = model.solve()

# Update the best solution
if best_solution is None or best_fitness < best_solution["fit"]:
    best_solution = {
        "fit": best_fitness,
        "position": best_position
    }

current_iteration += 1
trainings_epochs.append({"epoch": current_iteration,
                        "trainings": trainings_duration, "best_solution": best_solution})
trainings_duration = []

# Increment the iteration count
# Save the session periodically
with open(session_file, 'wb') as f:
    pickle.dump({
        "model": model,
        "best_solution": best_solution,
        "current_iteration": current_iteration
    }, f)


###########################################################


--------------------------2/0--------------------------------


###########################################################



Instructions for updating:
Colocations handled automatically by placer.


Epoch Training Started at : 2023-06-19 22:27:14.863449
currently working one voxel : 1, voxels left : 1 
Voxel Training 1 Started at : 2023-06-19 22:27:14.863773
Train on 249 samples
Epoch 1/30


2023-06-19 22:27:46.079919: W tensorflow/core/common_runtime/bfc_allocator.cc:343] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch sizes to observe the performance impact. Set TF_ENABLE_GPU_GARBAGE_COLLECTION=false if you'd like to disable this feature.
2023-06-19 22:27:46.259898: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.17GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-06-19 22:27:46.265049: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.19GiB 

249/249 [==============================] - 31s 123ms/sample - loss: 0.2150 - _calculate_reconstruction_loss: 0.0919 - _calculate_porosity_loss: 0.0936 - _calculate_kl_loss: 25.1850
Epoch 2/30
249/249 [==============================] - 10s 39ms/sample - loss: 0.0558 - _calculate_reconstruction_loss: 0.0294 - _calculate_porosity_loss: 0.0052 - _calculate_kl_loss: 20.8037
Epoch 3/30
249/249 [==============================] - 11s 42ms/sample - loss: 0.0531 - _calculate_reconstruction_loss: 0.0277 - _calculate_porosity_loss: 0.0050 - _calculate_kl_loss: 20.1046
Epoch 4/30
249/249 [==============================] - 10s 41ms/sample - loss: 0.0521 - _calculate_reconstruction_loss: 0.0272 - _calculate_porosity_loss: 0.0045 - _calculate_kl_loss: 19.8187
Epoch 5/30
249/249 [==============================] - 10s 38ms/sample - loss: 0.0494 - _calculate_reconstruction_loss: 0.0269 - _calculate_porosity_loss: 0.0042 - _calculate_kl_loss: 18.1058
Epoch 6/30
249/249 [==============================] - 1

...

Epoch Training Started at : 2023-06-19 23:57:01.125091
currently working one voxel : 1, voxels left : 1 
Voxel Training 1 Started at : 2023-06-19 23:57:01.125253
Train on 249 samples
Epoch 1/40
249/249 [==============================] - 8s 31ms/sample - loss: 0.5429 - _calculate_reconstruction_loss: 0.2137 - _calculate_porosity_loss: 0.2120 - _calculate_kl_loss: 113.9873
Epoch 2/40
249/249 [==============================] - 2s 9ms/sample - loss: 0.3806 - _calculate_reconstruction_loss: 0.1062 - _calculate_porosity_loss: 0.1604 - _calculate_kl_loss: 113.2668
Epoch 3/40
249/249 [==============================] - 3s 11ms/sample - loss: 0.3292 - _calculate_reconstruction_loss: 0.0817 - _calculate_porosity_loss: 0.1329 - _calculate_kl_loss: 113.9762
Epoch 4/40
249/249 [==============================] - 3s 13ms/sample - loss: 0.2939 - _calculate_reconstruction_loss: 0.0687 - _calculate_porosity_loss: 0.1134 - _calculate_kl_loss: 111.4957
Epoch 5/40
249/249 [==============================] - 

Epoch Training Started at : 2023-06-20 00:01:21.719800
currently working one voxel : 1, voxels left : 1 
Voxel Training 1 Started at : 2023-06-20 00:01:21.719962
Train on 249 samples
Epoch 1/40
249/249 [==============================] - 8s 33ms/sample - loss: 0.5623 - _calculate_reconstruction_loss: 0.2188 - _calculate_porosity_loss: 0.2265 - _calculate_kl_loss: 113.6742
Epoch 2/40
249/249 [==============================] - 3s 10ms/sample - loss: 0.3958 - _calculate_reconstruction_loss: 0.1075 - _calculate_porosity_loss: 0.1743 - _calculate_kl_loss: 112.7988
Epoch 3/40
249/249 [==============================] - 3s 10ms/sample - loss: 0.3344 - _calculate_reconstruction_loss: 0.0787 - _calculate_porosity_loss: 0.1408 - _calculate_kl_loss: 114.2982
Epoch 4/40
249/249 [==============================] - 3s 11ms/sample - loss: 0.2927 - _calculate_reconstruction_loss: 0.0632 - _calculate_porosity_loss: 0.1161 - _calculate_kl_loss: 112.8373
Epoch 5/40
249/249 [==============================] -

Epoch Training Started at : 2023-06-20 00:05:32.301223
currently working one voxel : 1, voxels left : 1 
Voxel Training 1 Started at : 2023-06-20 00:05:32.301404
Train on 249 samples
Epoch 1/40
192/249 [======================>.......] - ETA: 2s - loss: 0.3017 - _calculate_reconstruction_loss: 0.1146 - _calculate_porosity_loss: 0.0851 - _calculate_kl_loss: 101.9495

2023-06-20 00:06:10.206626: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 128.00MiB (rounded to 134217728)requested by op training/RMSprop/gradients/gradients/AddN_2-0-TransposeNHWCToNCHW-LayoutOptimizer
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-06-20 00:06:10.215902: W tensorflow/core/common_runtime/bfc_allocator.cc:474] ******x************************_****x*********************x***********************x******x**********
2023-06-20 00:06:10.218610: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at transpose_op.cc:183 : RESOURCE_EXHAUSTED: OOM when allocating tensor with shape[32,16,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc


ResourceExhaustedError: 2 root error(s) found.
  (0) RESOURCE_EXHAUSTED: OOM when allocating tensor with shape[32,16,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node training/RMSprop/gradients/gradients/AddN_2-0-TransposeNHWCToNCHW-LayoutOptimizer}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[training/RMSprop/gradients/gradients/generate/batch_normalization_20/cond_grad/StatelessIf/then/_4262/gradients/FusedBatchNormV3_grad/FusedBatchNormGradV3/_3717]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

  (1) RESOURCE_EXHAUSTED: OOM when allocating tensor with shape[32,16,256,256] and type float on /job:localhost/replic

In [ ]:
# print("\n".join( [str(training) for epoch in trainings_epochs for training in epoch]))
print(f"Best solution : {best_solution['fit']}")
batch_size, learning_rate, opt, epoch, num_filters, latent_space_dim, reduced_dim = decode_solution(
    best_solution["position"])
print(f"Batch size : {batch_size}, learning_rate : {learning_rate}, opt : {opt}, epoch : {epoch}, num_filters : {num_filters}, latent_space_dim : {latent_space_dim}, reduced_dim : {reduced_dim}")

Best solution : 0.01349127304975886

Batch size : 8, learning_rate : 0.0045351798887080024, opt : RMSprop, epoch : 40, num_filters : 8, latent_space_dim : 64, reduced_dim : 128

In [ ]:
current_iteration

### Save History

In [ ]:
model.history.save_global_objectives_chart(filename="hello/goc")
model.history.save_local_objectives_chart(filename="hello/loc")

model.history.save_global_best_fitness_chart(filename="hello/gbfc")
model.history.save_local_best_fitness_chart(filename="hello/lbfc")

model.history.save_runtime_chart(filename="hello/rtc")

model.history.save_exploration_exploitation_chart(filename="hello/eec")

model.history.save_diversity_chart(filename="hello/dc")

model.history.save_trajectory_chart(
    list_agent_idx=[3, 5], selected_dimensions=[3], filename="hello/tc")

### Remove old sessions

In [ ]:
if os.path.exists(session_file):
    os.remove(session_file)

In [19]:
print(len(trainings_duration))
trainings_epochs.append({"epoch": current_iteration,
                        "trainings": trainings_duration, "best_solution": best_solution})
trainings_duration = []

17


In [20]:
for training in trainings_epochs:
    print(len(training["trainings"]))

17


In [21]:
import json
jd = []
for training in trainings_epochs:
    json_data = []
    for obj in training["trainings"]:
        training_obj = obj.__dict__
        samples = []
        for sample in training_obj["samples"]:
            samples.append(sample.__dict__)
        training_obj["samples"] = samples
        json_data.append(training_obj)
    jd.append(json_data)

# Print the JSON data
print(jd)
# Convert the object to JSON
with open("output.json", "w") as file:
    file.write(json.dumps(jd))

[[{'title': 'Epoch Training', 'started': 1687210034.8634493, 'params': {'inputShape': (256, 256, 1), 'latent_space_dim': 64, 'reducedDimension': 128, 'num_conv_layers': 8, 'learning_rate': 0.0012699819863385329, 'batch_size': 8, 'epochs': 30, 'opt': 'Adam', 'loss': 0.01343773884407009}, 'ended': 1687210721.0356452, 'samples': [{'title': 'Voxel Training 1', 'started': 1687210034.863773, 'params': {}, 'ended': 1687210406.466543}, {'title': 'Voxel Training 2', 'started': 1687210406.4668467, 'params': {}, 'ended': 1687210721.0354059}]}, {'title': 'Epoch Training', 'started': 1687210728.7096472, 'params': {'inputShape': (256, 256, 1), 'latent_space_dim': 256, 'reducedDimension': 128, 'num_conv_layers': 7, 'learning_rate': 0.008196078947439053, 'batch_size': 16, 'epochs': 30, 'opt': 'SGD', 'loss': 0.10108081338156658}, 'ended': 1687210976.3239217, 'samples': [{'title': 'Voxel Training 1', 'started': 1687210728.7097895, 'params': {}, 'ended': 1687210862.921002}, {'title': 'Voxel Training 2', 

## Testing the model

### Load test data

In [ ]:
X_test = np.load('X_test.npy')
X_test.shape

In [ ]:
first = X_test[0]
Topredict = first[1:]
test = first[:-1]

In [ ]:
n = 10

plt.figure(figsize=(20, 4))
for i in range(n):
    # original
    ax = plt.subplot(2, n, i+1)
    plt.imshow(Topredict[i].astype("float32"))
    plt.title('original')
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # reconstructed
    ax = plt.subplot(2, n, i+1+n)
    plt.imshow(testing[i].reshape(256, 256, 1))
    plt.title('reconstructed')
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

In [ ]:
# reconstruction.Generate.save('reconstruction_10batch_10epochs.h5')

In [ ]:
file_path = "reconstructed.raw"
testing.insert(0, first[0].reshape(1, 256, 256, 1))
print(len(testing))

In [ ]:
final = np.array(testing, np.float32).reshape(250, 256, 256)
final.shape

In [ ]:
file_path = "my_array.npy"
np.save("file", final)

In [ ]:
final.tofile(file_path)

In [ ]:
Volume = np.fromfile(file_path, dtype=np.uint8)
Volume = Volume.reshape(250, 256, 256)
Volume.shape